# 基于二项分布的双爆配比分析——1：2真的是最佳配比吗？


## 1. 从期望的角度出发，但是继续深入
假设:可分配的总双爆词条数相同,即总和一致.而在原神的圣遗物、武器词条设计下,词条数一定,可以分配的暴击和暴伤比为1:2.由基本不等式可以得到,在连续情况下,伤害期望的最大值正是在暴击和暴伤的比为1:2时取得.这本身没有任何问题,但是,这里求得的是伤害期望.对于一次具体的深渊实战,由于攻击的次数并不是无穷大,一次战斗伤害的总值实际上服从一个特定的随机分布.对于一次攻击是否暴击,可以看做是一次伯努利试验,而一场战斗实际上是多次重复的伯努利试验.因此,暴击次数可以看做是符合二项分布.再结合暴击伤害值,通过程序模拟(懒得算)可以得到具体暴击次数(总伤害)的分布情况.

## 2. 研究分布情况,怎么帮助满星深渊
为了便于进行研究,首先抽象出一个问题的模型.这里只计算伤害的数值对于战斗的影响,排除其他因素.并且将伤害的计算简单化,且只考虑双爆因素.

这里先给出一个具体的例子.加入一个面板可以达到100/200的角色(比如220分+夜兰)，那么他的伤害可以计算为100 + 1 * 200 = 300.并且由于暴击率达到了1,他的伤害不会有任何的浮动.假设用于,满星深渊的时间可以打出100次攻击,那么造成的伤害就是30000,并且这个值不会变化.
但是,如果满星深渊所需要的伤害是30500,甚至31000,这样的角色将永远不可能达到这个要求.

这里这个角色完美遵循了双爆1:2的计算结果配面板,但是却不是最佳的情况.这里如果想要打满深渊,只能采用凹分常见操作,也就是降暴击,比如90/220,来尝试达到30500的伤害量.那么,达成这个值有多困难,压更低的暴击会不会更好.在更低双爆的面板下,如果打不满深渊,降爆会不会是一种好的策略呢？

In [6]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from scipy import stats

In [11]:
##达成30500伤害量，220爆伤需要94次暴击
stats.binom.cdf(k = 100,n = 100, p =0.9) - stats.binom.cdf(k = 94,n = 100, p =0.9)
##90暴击的角色在100次攻击中暴击94次及以上的概率

0.05757688648703385

In [14]:
##达成30500伤害量，240爆伤需要86次暴击
stats.binom.cdf(k = 100,n = 100, p =0.8) - stats.binom.cdf(k = 86,n = 100, p =0.8)
##80暴击的角色在100次攻击中暴击86次及以上的概率

0.04691223716078596

In [18]:
##达成31000伤害量，240爆伤需要96次暴击
stats.binom.cdf(k = 100,n = 100, p =0.9) - stats.binom.cdf(k = 96,n = 100, p =0.9)
##90暴击的角色在100次攻击中暴击96次及以上的概率

0.007836487121184454

In [19]:
##达成31000伤害量，240爆伤需要88次暴击
stats.binom.cdf(k = 100,n = 100, p =0.8) - stats.binom.cdf(k = 88,n = 100, p =0.8)
##80暴击的角色在100次攻击中暴击88次及以上的概率

0.01257487634812171

## 3. 从具体的值出发，降暴击是否可行
本部分有两个目标：

1.通过对代入具体数据的高双爆和低双爆的情况(由于暴击存在上限，猜测可能会出现差异).分析一下达到某个特定伤害值.在各自赛道不同暴击值的概率.在什么特定需求下,更需要降低暴击率?

2.对于一个特定的伤害阈值和提升暴伤占比的面板,需要多少凹度才能达到要求.

In [21]:
x = np.arange(0,101)
crit08 = stats.binom(100,0.8)
crit8 = crit08.pmf(x)
crit09 = stats.binom(100,0.9)
crit9 = crit09.pmf(x)

damage08 = np.sum(crit8 * x * 2.4) + 100  #80/240
damage09 = np.sum(crit9 * x * 2.2) + 100  #90/220
print(round(damage08,0))
print(round(damage09,0))

for i in range(80,101):
    chance8 = stats.binom.cdf(k = 100,n = 100, p =0.8) - stats.binom.cdf(k = i -1,n = 100, p =0.8)
    damage08high = ( np.sum(crit8[i:101]/chance8 * x[i:201] * 2.4) + 100 )
    print(i,chance8,damage08high)
print('\n')


for i in range(85,101):
    chance9 = stats.binom.cdf(k = 100,n = 100, p =0.9) - stats.binom.cdf(k = i -1,n = 100, p =0.9)
    damage09high = ( np.sum(crit9[i:101]/chance9 * x[i:201] * 2.2) + 100 )
    print(i,chance9,damage09high)

292.0
298.0
80 0.5594615848733986 298.81571058989084
81 0.46016137006457336 300.2865022940186
82 0.3620870838336354 301.88091020487343
83 0.2711889648878879 303.58394845477494
84 0.19233758459760097 305.3812071003156
85 0.12850551483879724 307.25943213760365
86 0.08044372113805087 309.20680792732185
87 0.04691223716078596 311.213031478752
88 0.02532875322139705 313.26925689623147
89 0.01257487634812171 315.367969197102
90 0.005696380955793501 317.50282781530336
91 0.0023335609862108475 319.6685048291042
92 0.0008553983622184846 321.86053197054594
93 0.00027698690065613985 324.0751632245427
94 7.796360205414299e-05 326.3092553798582
95 1.8680066300347242e-05 328.56016637728646
96 3.70317305731227e-06 330.82567001265215
97 5.829869649254249e-07 333.10388505356985
98 6.831709409294007e-08 335.3932164224556
99 5.296293514867045e-09 337.6923087515708
100 2.0370360953592126e-10 339.999985976718
85 0.9601094728889167 298.6365540708988
86 0.9274270347351196 299.0466251002105
87 0.8761232074006

In [41]:
x = np.arange(0,201)
crit08 = stats.binom(200,0.8)
crit8 = crit08.pmf(x)
crit09 = stats.binom(200,0.9)
crit9 = crit09.pmf(x)

damage08 = np.sum(crit8 * x * 2.4) + 100  #80/240
damage09 = np.sum(crit9 * x * 2.2) + 100  #90/220
print(damage08)
print(damage09)

for i in range(160,201):
    chance8 = stats.binom.cdf(k = 200,n = 200, p =0.8) - stats.binom.cdf(k = i -1,n = 200, p =0.8)
    damage08high = ( np.sum(crit8[i:201]/chance8 * x[i:201] * 2.4) /2 + 100 )
    print(i/2,chance8,damage08high)
print('\n')


for i in range(170,201):
    chance9 = stats.binom.cdf(k = 200,n = 200, p =0.9) - stats.binom.cdf(k = i -1,n = 200, p =0.9)
    damage09high = ( np.sum(crit9[i:201]/chance9 * x[i:201] * 2.2) /2 + 100 )
    print(i/2,chance9,damage09high)

484.0000000000001
496.00000000000017
80.0 0.5421796450494936 296.98394328867505
80.5 0.4718100460078052 297.72728924715636
81.0 0.4018775252210365 298.5151032688793
81.5 0.33453509779673996 299.3434798579219
82.0 0.2717372513642671 300.20858934166273
82.5 0.215066024095938 301.10677371112683
83.0 0.1656074984799416 302.0346087945935
83.5 0.12389548892428204 302.988939175411
84.0 0.08992642725021194 303.9668923445636
84.5 0.06323645022058544 304.9658779310039
85.0 0.043021556375661296 305.983576874157
85.5 0.028276574982893043 307.0179243550049
86.0 0.017929219619546855 308.0670893339662
86.5 0.010950770653569197 309.12945272434695
87.0 0.006432930860566399 310.2035855775256
87.5 0.003628754437323245 311.28822816012035
88.0 0.0019622724486529908 312.3822704413641
88.5 0.0010154076823630787 313.48473425427443
89.0 0.0005018539108160303 314.594757221914
89.5 0.00023642162439840853 315.7115784286165
90.0 0.00010592977409262794 316.83452574774964
90.5 4.503357728313784e-05 317.9630047019773

C:\Users\Tan90plays\AppData\Local\Temp\ipykernel_14056\2799712376.py:14: RuntimeWarning: divide by zero encountered in true_divide
  damage08high = ( np.sum(crit8[i:201]/chance8 * x[i:201] * 2.4) /2 + 100 )


In [40]:
times = 100
x = np.arange(0,times + 1)
crit08 = stats.binom(times,0.8)
crit8 = crit08.pmf(x)
crit09 = stats.binom(times,0.9)
crit9 = crit09.pmf(x)

damage08 = np.sum(crit8 * x * 2.4)/times * 100 + 100  #80/240
damage09 = np.sum(crit9 * x * 2.2)/times * 100 + 100  #90/220
print(round(damage08,0))
print(round(damage09,0))

for i in range(int(0.8 * times) ,times + 1):
    chance8 = stats.binom.cdf(k = times,n = times, p =0.8) - stats.binom.cdf(k = i -1,n = times, p =0.8)
    damage08high = ( np.sum(crit8[i:times+1]/chance8 * x[i:times+1] * 2.4) /times * 100 + 100 )
    print(round(i/times * 100,1),round(chance8,6),round(damage08high,3),)
print('\n')

for i in range(int(0.85 * times) ,times+1):
    chance9 = stats.binom.cdf(k = times,n = times, p =0.9) - stats.binom.cdf(k = i -1,n = times, p =0.9)
    damage09high = ( np.sum(crit9[i:times+1]/chance9 * x[i:times+1] * 2.2) /times * 100 + 100 )
    print(round(i/times * 100,1),round(chance9,6),round(damage09high,3))

292.0
298.0
80.0 0.559462 298.816
81.0 0.460161 300.287
82.0 0.362087 301.881
83.0 0.271189 303.584
84.0 0.192338 305.381
85.0 0.128506 307.259
86.0 0.080444 309.207
87.0 0.046912 311.213
88.0 0.025329 313.269
89.0 0.012575 315.368
90.0 0.005696 317.503
91.0 0.002334 319.669
92.0 0.000855 321.861
93.0 0.000277 324.075
94.0 7.8e-05 326.309
95.0 1.9e-05 328.56
96.0 4e-06 330.826
97.0 1e-06 333.104
98.0 0.0 335.393
99.0 0.0 337.692
100.0 0.0 340.0


85.0 0.960109 298.637
86.0 0.927427 299.047
87.0 0.876123 299.623
88.0 0.801821 300.385
89.0 0.703033 301.339
90.0 0.583156 302.477
91.0 0.45129 303.785
92.0 0.320874 305.243
93.0 0.206051 306.827
94.0 0.117156 308.517
95.0 0.057577 310.293
96.0 0.023711 312.14
97.0 0.007836 314.044
98.0 0.001945 315.994
99.0 0.000322 317.982
100.0 2.7e-05 320.0


In [42]:
times = 100
CritRate1 = 0.6
CritRate2 = 0.7
CritDamage1 = 160
CritDamage2 = 140
range1_h = 0.6
range1_t = 0.8
range2_h = 0.65
range2_t = 0.85
CritDamage1 = CritDamage1 / 100
CritDamage2 = CritDamage2 / 100

x = np.arange(0,times + 1)
crit01 = stats.binom(times,CritRate1)
crit1 = crit01.pmf(x)
crit02 = stats.binom(times,CritRate2)
crit2 = crit02.pmf(x)

damage01 = np.sum(crit1 * x * CritDamage1)/times * 100 + 100  
damage02 = np.sum(crit2 * x * CritDamage2)/times * 100 + 100  
print(round(damage01,0))
print(round(damage02,0))

for i in range(int(range1_h * times) ,int(range1_t * times) + 1):
    chance1 = stats.binom.cdf(k = times,n = times, p =CritRate1) - stats.binom.cdf(k = i -1,n = times, p =CritRate1)
    damage01high = ( np.sum(crit1[i:times+1]/chance1 * x[i:times+1] * CritDamage1) /times * 100 + 100 )
    print(round(i/times * 100,1),round(chance1,6),round(damage01high,3))
print('\n')

for i in range(int(range2_h * times) ,int(range2_t * times) + 1):
    chance2 = stats.binom.cdf(k = times,n = times, p =CritRate2 ) - stats.binom.cdf(k = i -1,n = times, p =CritRate2 )
    damage02high = ( np.sum(crit2[i:times+1]/chance2 * x[i:times+1] * CritDamage2) /times * 100 + 100 )
    print(round(i/times * 100,1),round(chance2,6),round(damage02high,3))

196.0
198.0
60.0 0.543294 201.741
61.0 0.462075 202.75
62.0 0.382188 203.826
63.0 0.30681 204.963
64.0 0.238611 206.152
65.0 0.179469 207.389
66.0 0.130337 208.666
67.0 0.091254 209.979
68.0 0.061504 211.324
69.0 0.039848 212.695
70.0 0.024783 214.091
71.0 0.014775 215.507
72.0 0.008433 216.941
73.0 0.0046 218.391
74.0 0.002396 219.855
75.0 0.001189 221.331
76.0 0.000562 222.818
77.0 0.000252 224.315
78.0 0.000107 225.82
79.0 4.3e-05 227.333
80.0 1.6e-05 228.852


65.0 0.883921 199.445
66.0 0.837142 199.917
67.0 0.779258 200.475
68.0 0.710719 201.119
69.0 0.633108 201.844
70.0 0.549124 202.646
71.0 0.46234 203.519
72.0 0.376778 204.454
73.0 0.296366 205.445
74.0 0.224399 206.486
75.0 0.16313 207.57
76.0 0.11357 208.691
77.0 0.075531 209.845
78.0 0.047866 211.027
79.0 0.028831 212.234
80.0 0.016463 213.462
81.0 0.008887 214.707
82.0 0.004523 215.969
83.0 0.002163 217.245
84.0 0.000969 218.532
85.0 0.000405 219.831


In [26]:
stats.binom_test(1,25,0.012575) ##对次数的假设检验（n次里面至少出k次的概率）
##数据为80/240/31000

0.27120905777607063

In [48]:
stats.binom.ppf(0.95,100,0.6)  ##暴击次数在一定范围内的把握 ##也可用作出分次数在一定范围内的把握

68.0

## 小结
1.在需要达到更高伤害的时候(超过一定范围),降低暴击增加暴伤可以增加实现更高的概率达到目标伤害.

2.深渊有些时候要凹是有意义的，但是还是请提升练度（

In [26]:
?stats.binom